In [ ]:
!pip install openai
!pip install colorama
!pip install tqdm
!pip install datasets


In [3]:
# Import Necessary SDKs
import os
from openai import OpenAI
import json
import pandas as pd
from colorama import Fore, Style
import time
from tqdm import tqdm

In [10]:
token = "xxxxxxxxxxxxxxxxx-xxxxxxxxxxxxxxx-xxxxxxxxxxxxxxxxxxx"
endpoint = "https://models.inference.ai.azure.com"
model_name = "gpt-4o-mini"
csv_path = "grounding_truth.csv"

In [11]:
client = OpenAI(
    base_url=endpoint,
    api_key=token,
)

System_Prompt = """
Vous êtes un assistant puissant dans l'extraction d'informations a partir des factures francaises. \n
Observe attentivement la facture et extrait les informations suivantes:  \n

- Fournisseur : C'est le nom du fournisseur qui a vendu des produits ou des services à un client. Généralement, C'est dans le Logo du Fournisseur ou a côté. \n
- Date_Facture : C'est la date de la facture. \n
- Num_Facture : C'est le numéro de la facture se forme d'une série de numéros ou numéros avec lettres. \n
- TTC : C'est le montant total que le client va payer, peut etre referer avec : Total TTC, Net à Payer ou quelque chose similaire. \n
- TT  : C'est le montant total de la facture hors taxes, peut etre referer avec : Total HT ou quelque chose similaire. \n
- TVA : C'est le montant de TVA appliqué au Total hors taxes, peut etre referer avec : TVA, Total TVA, Taxe ou quelque chose similaire. \n

- Votre reponse est un String se la forme d'un dictionaire python avec les clés-valeurs suivantes : \n

     "Invoice_Name": le nom et format du l'image du la facture,
     "Fournisseur": "Fournisseur",
     "Date_Facture": "Date_Facture",
     "Num_Facture": "Num_Facture",
     "TTC": "TTC",
     "TT": "TT",
     "TVA": "TVA"

- Ne pas formater votre reponse comme un code python , repondre juste en texte. \n
- Les 3 prix TT,TTC,TVA doivent être des chiffres sans indication de devise. \n
- le nom de Fournisseur doit être en majuscule. \n
"""

Invoice_Names = [
    "109.jpg", "114.jpg", "121.jpg", "129.jpg", "131.jpg", "151.jpg", "164.jpg",
    "2.jpg", "3.jpg", "307.jpg", "311.jpg", "314.jpg", "315.jpg", "316.jpg",
    "317.jpg", "319.jpg", "327.jpg", "334.jpg", "340.jpg", "383.jpg", "417.jpg",
    "419.jpg", "49.jpg", "57.jpg", "58.jpg", "61.jpg", "66.jpg", "67.jpg",
    "72.jpg", "76.jpg"
]


In [13]:
def Save_to_csv(json_data):
    data = json.loads(json_data)
    df = pd.DataFrame([data])

    if os.path.exists(csv_path):
        df.to_csv(csv_path, mode='a', header=False, index=False)
    else:
        df.to_csv(csv_path, index=False)

    print(Fore.GREEN + Style.BRIGHT + "CSV Updated Successfully" + Style.RESET_ALL)


def Run_ALL(Names):
  counter = 0
  for Name in tqdm(Names,desc="Processing Invoices", unit="number"):
        Hugg_path = "https://huggingface.co/datasets/Noureddinesa/Images_Test/resolve/main/"+Name
        response = client.chat.completions.create(
                      model=model_name,
                      messages=[
                          {"role": "system", "content": System_Prompt},
                          {
                          "role": "user",
                          "content": [
                            {"type": "text", "text": f"Extraire les Informations demandées a partir de la facture {Name} suivante : "},
                            {
                              "type": "image_url",
                              "image_url": {"url":Hugg_path},
                            },
                          ],
                          }
                            ],
                      #max_tokens=500,
                    )

        data = response.choices[0].message.content
        print(data)
        Save_to_csv(data)
        # Sleep time between requests of 2 seconds
        time.sleep(2)
        counter += 1
        # We set a Sleep of 1 min so we don't exceed the rate limit (10 requests/min)
        if counter == 7:
          time.sleep(60)
          counter = 0

In [ ]:
Run_ALL(Invoice_Names)